# Análise de Estatística Descritiva dos Dados do Brasileirão

## 01 - Setup

In [19]:
def get_separador() -> str:
  sistema_operacional = platform.system()
  separador = ""

  if sistema_operacional == "Windows":
    separador = "\\"
  elif sistema_operacional == "Linux":
    separador = "/"
  
  return separador

# Importação das bibliotecas
import pandas as pd
import platform
import os

# Pegando o arquivo .csv do dataset
separador = get_separador()
caminho_atual = os.getcwd()
caminho_dataset = os.path.join(caminho_atual, f"..{separador}dataset{separador}brasileirao.csv")

# Este dataset foi baixado contendo os dados de jogos ocorridos até o dia 13/05/2025
brasileirao = pd.read_csv(caminho_dataset)

## 02 - Análises

### 02.1 - Desempenho médio de cada time por temporada

Crie tabelas, cujas linhas são os _times_, as colunas são as _temporadas_ e a variável em análise é a _quantidade de pontos_, para as seguintes informações:
  - Média
  - Mediana
  - Desvio Padrão
  - Máximo
  - Mínimo

Destaque o ano com maior variação

Numa disputa entre dois times:
  - Em caso de vitória, o vencedor ganha 3 pontos.
  - Em caso de empate, ambos ganham 1 ponto.

In [ ]:
def verifica_presenca_time(time: str, temporada: int) -> bool:
  jogos_time_temporada = brasileirao[
    (brasileirao["Season"] == temporada) &
    (brasileirao["Home"] == time)
  ]

  return not (jogos_time_temporada.empty)

def calcula_pontos_time(time: str, temporada: int) -> int:

  if not verifica_presenca_time(time, temporada):
    return 0
  
  jogos_time_temporada = brasileirao[
    (brasileirao["Season"] == temporada) &
    ((brasileirao["Home"] == time) | (brasileirao["Away"] == time))
  ]

  vitorias_casa = jogos_time_temporada[
    (jogos_time_temporada["Home"] == time) & 
    (jogos_time_temporada["Res"] == "H")
  ]

  vitorias_visitante = jogos_time_temporada[
    (jogos_time_temporada["Away"] == time) & 
    (jogos_time_temporada["Res"] == "A")
  ]

  empates = jogos_time_temporada[(jogos_time_temporada["Res"] == "D")]

  total_pontos = ((len(vitorias_casa) + len(vitorias_visitante)) * 3) + (len(empates) * 1)

  return total_pontos

def get_times_temporada(temporada: int) -> set[str]:
  tabela_temporada = brasileirao[(brasileirao["Season"] == temporada)]
  times = set(tabela_temporada["Home"])
  return times

def get_todos_times() -> set[str]:
  temporadas = set(brasileirao.Season)
  times = set()

  for t in temporadas:
    times_temporada = get_times_temporada(t)
    times = times.union(times_temporada)
  
  return times

def calcula_desempenho_pontos_totais() -> pd.DataFrame:
  temporadas = ["Time"] + sorted(list(set(brasileirao.Season)))
  times = sorted(get_todos_times())
  dados = list()

  for t in times:
    pontos_campeonato = [t] 
    
    for temp in temporadas[1:]:
      pontos = calcula_pontos_time(t, temp)
      pontos_campeonato.append(pontos)
    
    dados.append(pontos_campeonato)

  desempenho_pontos_totais = pd.DataFrame(data=dados, columns=temporadas)
  return desempenho_pontos_totais

calcula_desempenho_pontos_totais()



,Time,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,America MG,0,0,0,0,28,0,40,0,0,53,53,24,0,0
1,Athletico-PR,0,64,54,51,57,51,57,64,53,47,58,56,42,0
2,Atletico GO,30,0,0,0,0,36,0,0,50,53,36,0,30,0
3,Atletico-MG,72,57,62,69,62,54,59,48,68,84,58,66,47,12
4,Avai,0,0,0,42,0,43,0,20,0,0,35,0,0,0
5,Bahia,47,48,37,0,0,50,48,49,44,43,0,44,53,12
6,Botafogo RJ,55,61,34,0,59,53,51,43,27,0,53,64,79,11
7,Bragantino,0,0,0,0,0,0,0,0,53,56,44,62,44,17
8,CSA,0,0,0,0,0,0,0,32,0,0,0,0,0,0
9,Ceara,0,0,0,0,0,0,44,39,52,50,37,0,0,12
